# Introduction to NeoRV32OnPynq


## Aims

* Introduce the compiler magic provided by NeoRV32OnPynq, and how to use it. 
* Introduce the NeoRV32OnPynq overlay GUI

# Introducing the Compiler Magic


This section will introduce the compiler magic provided by NeoRV32OnPynq, riscvc.
This magic enables C programs to be written in any Jupiter Notebook cell, and then compiled simply by running said cell.
Before this magic can be used it must be loaded, which is done by the line below

In [ ]:
%load_ext NeoRV32OnPynq

The compiler magic is called using <code> %%riscvc <i>program_name</i> </code>. 
The compiled program can be found in "programs/<i>program_name</i>/<i>program_name</i>.bin", where "programs" is a folder in the same directory as the notebook that called the magic. 

The "programs" folder, and its children (each named for the program they contain), serve as a way to  prevent different programs from interferring with each other. Each child folder contains not only the compiled program (.bin), but also any intermedate files (.c, .o), and some files useful for debugging (.asm, .coe).

<div class="alert alert-box alert-info">
There is no need to worry about creating a "programs" folder, or any of its children, as the compiler magic handles this as needed.
</div>

<div class="alert alert-box alert-info">
The compiler magic is a cell magic, it must be the first statement in a cell, with the rest of the cell's content being interpreted by the magic.
</div>

## Compiler Magic Example

Below is an example of how to use the compiler magic.
Upon running the cell below a number of effects should be seen:
* If there wasn't already a "programs" folder, one will have been created
* If there wasn't already a "programs\flash_LEDs" folder, one will have been created
* A makefile was added to "programs\flash_LEDs"
* "programs\flash_LEDs\flash_LEDs.c" was (re)created and contains everything but the first line of the compiler magic cell 
* "programs\flash_LEDs\flash_LEDs.c.o" was (re)created
* "programs\flash_LEDs\flash_LEDs.bin" was (re)created
* "programs\flash_LEDs\flash_LEDs.asm" was (re)created
* "programs\flash_LEDs\flash_LEDs.coe" was (re)created


In [ ]:
%%riscvc flash_LEDs

//Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define offsets for LED GPIO handler's register map
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;
    
    // Set the LEDs in off state
    int32_t LED_data = 0;

    // Toggle the LEDs forever
    while(1)
    {
        LEDs[value] = LED_data;
        if (LED_data != 0)
        {
            LED_data = 0;
        }
        else
        {
            LED_data = 15;
        }
    }

    return 0;
}



## Compiler Magic Example 2


A notebook is not restricted to compiling a single program, below is an example of the compiler magic being used to compile another program, the same set of effects should be seen upon running this cell.

In [ ]:
%%riscvc LED_counter

//Define int32_t
#include <stdint.h>

// Define volatile pointer to LED GPIO handler
int32_t volatile * const LEDs = (int32_t*) 0x40020000;

// Define offsets for LED GPIO handler's register map
const int32_t value = 0;
const int32_t tristate = 1;

int main()
{
    // Set all LEDs GPIO as outputs
    LEDs[tristate] = 0;

    // Toggle the LEDs forever
    while(1)
    {
        for(int32_t i=0; i < 16; i++)
        {
            LEDs[value] = i;
        }
    }

    return 0;
}

# Interacting with the Processor


Now that we have some programs compiled with the compiler magic, lets look at how to interact with the the NeoRV processor.
All interactions with the processor are done through the `NeoRV32` class within the `NeoRV32OnPynq` module.
The code below first imports the `NeoRV32` class and then by creating an object of the `NeoRV32` class sets up the board with the NeoRV processor overlay.

In [ ]:
from NeoRV32OnPynq import NeoRV32
neorv = NeoRV32()

The recommended method of interacting with the processor is via the GUI provided; which consists of 3  parts, each handling a different stage/type of interaction.
Each part will be covered in their own section, but below is an overview of each.
* Program Loading, this section handles the selection and loading of programs to run on the processor
* Execution Control, this section controls when the processor can executes instruction, providing stepping, continous, and run until condition options
* Internal Inspection, this section displays a range of signals internal to the NeoRV32, aiding in debugging and/or tracking of the instructions through the processor

## Program Loading


The program loading section of the GUI is responsbile for enabling a user to select any of the programs found in "programs" and loading them to be run by the NeoRV32. 
It is accessed via the `load_program` function of the `NeoRV32` class as shown below.

In [ ]:
neorv.load_program()

To load a program to the processor
<ol>
<li>Select the program from the dropdown. If the program you are looking isn't listed in the dropdown, try rerunning `reorv.load_program()` this will check the "programs" folder again and update the program list</li>
<li>Once the desired program is selected in the dropdown, click the "Load Program" button. This will read that programs .bin file, load it to the processor, and reset the processor's PC back to the start of the new program</li>
<li>Once the program has been loaded the "Last Loaded" field will update to show the name of the program loaded, and ready to be executed.</li>
</ol>

## Execution Control

The execution control section of the GUI is responsbile for controlling when the processor can execute instructions. 
It enables control of the NeoRV32 as varied as running in single clock cycle steps to running the processor until a specific position within the program is reached.
It is accessed via the `execution_control` function of the `NeoRV32` class as shown below.

In [ ]:
 neorv.execution_control()

The execution control GUI has 3 main sections: the rules controls, the buttons, and the output panel.
I will explain the buttons first as it is only when they are clicked that anything is sent to the processor.
<ul>
<li>Step Clock, when clicked the processor runs for a single clock cycle</li>
<li>Step Instruction, when clicked the process runs until an instruction completes.</li>
<li>Run/Stop, the behaviour of this button depends on the rules control section.
<ul>
<li> If none of the rules are enabled, then the processor's state is checked. 
If the processor is running, it is stopped, while if it wasn't running, it is set to run continually</li>
<li> If any combination of the rules are enabled, then the processor is set to run as dictated by the rules</li>
</ul>
</li>
<li>Restart Program, when clicked the processor is reset back to the start of program, and stopped before any cycles are executed</li>
</ul>

There are currently 3 rules:
<ul>
<li> Clock Counter, runs the processor for the given number of clock cycles </li>
<li> Instructions Counter, runs the processor until the given number of instructions have been completed </li>
<li> PC Target, runs the processor until its program counter (PC) equals the given value</li>
</ul>
These rules can be used in any combination, however when more than 1 are enabled at the same time the processor will only run until any of them complete. 
Eg if the clock counter rule is enabled with a count of 100 clock cycles, at same time as the PC target rule is enabled.
And it only takes 20 clock cycles to reach the given PC target, then the processor will stop upon reaching the PC target.
Whereas if it would take 150 clock cycles to reach the given PC target, then the processor will stop after the 100 clock cycle count is reached.

Upon being clicked all of the buttons will update the output panel with how they are telling the processor to run.


<div class="alert alert-box alert-info">
Note: An instruction completing is determined by the processor's PC updating.
This means that if the processor has partially executed an instruction (say because it was stepped by clock cycles) when an instruction based rule is started, finishing the partially executed instruction will count as the first instruction; with the first full instruction counting as the second instruction.
</div>

<div class="alert alert-box alert-info">
All the rules' number fields support binary (prefixed with 0b/0B), hexidecimal (prefixed with 0x/0X), and decimal (no prefix) numbers
</div>

## Internal Inspection

The internal inspection section of the GUI is responsbile for capturing and displaying a range of signals internal to the processor. 
These signals are visible to aid in debugging, and to enable instructions to be tracked though the processor.
It is accessed via the `show_internals` function of the `NeoRV32` class as shown below.

In [ ]:
neorv.show_internals()

The internal inspection GUI only has a single button, labeled "Capture Internals",  which when clicked will read the values of the internal signals and update the display with these new values.

The internal signals have been grouped by what part of the NeoRV/peripheral they relate to:
<ul>
<li> Regfile tab, signals under this tab relate to reading, writing, and comparing of NeoRV's registers</li>
<li> ALU tab, signals under this tab relate to the arithmetic logic unit (ALU) and handling the rv32i base instruction set</li>
<li> Co-Processors tab, signals under this tab relate the NeoRV's co-processors. 
Note co-processors are how the NeoRV handles RISC-V extensions, with a co-processor handling the instruction added by an extension</li>
<li> Bus Control, signals under this tab relate to controllong the instruction fetch and data access buses of the NeoRV</li>
<li> Bus Switch, signals under this tab relate to switching the internal bus between the instruction fetch and data access buses as needed by the NeoRV</li>
<li> External Bus, signals under this tab relate to converting the internal bus to an external axi4lite bus, via a intermedate wishbone bus</li>
<li> BRAM, the signals under this tab relate to the block RAM (BRAM) which acts as both instruction and data memory for the NeoRV</li>
<li> LED, the signals under this tab relate to controlling the 4 LEDs on the PYNQ-Z2 board with the NeoRV</li>
<li> Buttons, the signals under this tab relate to reading the 4 buttons on the PYnQ-Z2 board and making them readible to the NeoRV</li>
</ul>
</li>

<div class="alert alert-box alert-info">
Reading the internal signals takes multiple clock cycles, and so should be done only when the processor is stopped.
Otherwise the values captured and displayed may relate to different processor states, and carry little to no useful information.
</div>